In [3]:
from move_training_env import MoveTrainingEnv
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3 import PPO

import os
import numpy as np

In [ ]:
# Create environment instance
env = MoveTrainingEnv(
    robots="Panda",  # Use Panda robot
    gripper_types="default",
    has_renderer=True,  # Enable visualization
    has_offscreen_renderer=False,  # Disable offscreen rendering
    use_camera_obs=False,  # Don't use camera observations
    reward_shaping=False,  # Enable reward shaping
)

# Configurations
total_timesteps = 1000
#save_freq = 100
#save_path = os.path.dirname(os.path.abspath(__file__))
name_prefix = "ppo_panda_reach"

# # Initialize the checkpoint callback
# checkpoint_callback = CheckpointCallback(
#     save_freq=save_freq,
#     save_path=save_path,
#     name_prefix=name_prefix,
#     save_replay_buffer=True,
# )

# Define the model
model = PPO("MlpPolicy", env, verbose=1)

[robosuite INFO] Loading controller configuration from: c:\Users\xiaoyicong\anaconda3\envs\robosuite_project\lib\site-packages\robosuite\controllers\config\robots\default_panda.json (composite_controller_factory.py:121)


NameError: name '__file__' is not defined

In [ ]:

# Train the model
model.learn(total_timesteps=total_timesteps, callback=checkpoint_callback)
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
model.save(f"{os.path.dirname(os.path.abspath(__file__))}/{name_prefix}_{current_time}_{str(total_timesteps)}")
env.close()

exit()